In [1]:
using Plots
gr()

Plots.GRBackend()

In [4]:
using MLDataPattern
using LIBSVM

INFO: Precompiling module LIBSVM.


In [5]:
include("main.jl")

c

In [26]:
double_integrator = c.MPC.discretize(c.MPC.CTLinearSytstem([0. 1; 0 0], [0. 1]'), 0.1)
numresults = 1000
results = @time(c.collect_data(double_integrator, numresults));

 56.241376 seconds (1.83 M allocations: 752.844 MB, 0.36% gc time)


In [27]:
examples = convert(Vector{c.Example}, results);

labels = [e.best_action for e in examples]
data = hcat([vcat(e.contact, e.duals) for e in examples]...);

In [28]:
(data_train, labels_train), (data_test, labels_test) = splitobs((data, labels), 0.5);

In [29]:
@time model = svmtrain(data_train, labels_train; cost=100., probability=true)
@time predicted_labels, decision_values = svmpredict(model, data_test)

  0.449606 seconds (80 allocations: 1.355 MB)
  0.075870 seconds (23 allocations: 2.405 MB)


([4,1,5,4,2,8,16,1,9,2  …  10,6,5,5,1,1,3,5,5,2],
[0.0137477 0.0167142 … 0.00903094 0.00679276; 0.0312039 0.0501112 … 0.0547134 0.0174546; … ; 0.0723889 0.0389993 … 0.0326906 0.0395678; 0.0886511 0.00817799 … 0.00308116 0.00377413])

In [30]:
mean(predicted_labels .== labels_test)

0.5049900199600799

In [31]:
function topn(n)
    [model.labels[reverse(sortperm(decision_values[:, i]))[1:n]] for i in 1:size(decision_values, 2)]
end

topn (generic function with 1 method)

In [32]:
mean(getindex.(topn(1), 1) .== labels_test)

0.5049900199600799

In [33]:
t = topn(5)
mean([labels_test[i] in t[i] for i in 1:length(labels_test)])

0.872255489021956